In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime,timedelta

In [2]:
#Define the url for google news RSS feed(Ritch Site Summary)
url="https://news.google.com/rss?h1=en-US&gl=US&ceid=US:en"

In [8]:
#Make a request to the RSS feed URL
response=requests.get(url)
response

<Response [200]>

In [6]:
#Parse the XML content Using BeautifulSoup
soup=BeautifulSoup(response.content,"xml")

In [7]:
articles=soup.find_all("item")

In [9]:
articles=soup.find_all("item")

In [10]:
articles

[<item><title>Trump arrives in Miami for arraignment on first-ever federal charges against an ex-president - CNBC</title><link>https://news.google.com/rss/articles/CBMiY2h0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjMvMDYvMTIvZG9uYWxkLXRydW1wLWluZGljdG1lbnQtdHJ1bXAtYXJyaXZlcy1pbi1taWFtaS1mb3ItYXJyYWlnbm1lbnQuaHRtbNIBZ2h0dHBzOi8vd3d3LmNuYmMuY29tL2FtcC8yMDIzLzA2LzEyL2RvbmFsZC10cnVtcC1pbmRpY3RtZW50LXRydW1wLWFycml2ZXMtaW4tbWlhbWktZm9yLWFycmFpZ25tZW50Lmh0bWw?oc=5</link><guid isPermaLink="false">2128778852</guid><pubDate>Mon, 12 Jun 2023 21:49:44 GMT</pubDate><description>&lt;ol&gt;&lt;li&gt;&lt;a href="https://news.google.com/rss/articles/CBMiY2h0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjMvMDYvMTIvZG9uYWxkLXRydW1wLWluZGljdG1lbnQtdHJ1bXAtYXJyaXZlcy1pbi1taWFtaS1mb3ItYXJyYWlnbm1lbnQuaHRtbNIBZ2h0dHBzOi8vd3d3LmNuYmMuY29tL2FtcC8yMDIzLzA2LzEyL2RvbmFsZC10cnVtcC1pbmRpY3RtZW50LXRydW1wLWFycml2ZXMtaW4tbWlhbWktZm9yLWFycmFpZ25tZW50Lmh0bWw?oc=5" target="_blank"&gt;Trump arrives in Miami for arraignment on first-ever federal charge

In [11]:
data=[] #List to store scrapped data

In [13]:
#Calculate the cutoff time for thr past 24 hours
cutoff_time=datetime.now()-timedelta(hours=24)

In [17]:
for article in articles:
    pub_date=datetime.strptime(article.pubDate.text,"%a, %d %b %Y %H:%M:%S %Z")
    if pub_date>=cutoff_time:
        title=article.title.text
        link=article.link.text
        description=article.description.text
        data.append([title,link,description])
df=pd.DataFrame(data,columns=["Title","Link","Description"])

In [18]:
df.head()

Title   
0  Trump arrives in Miami for arraignment on firs...  \
1  California Gov. Newsom spars with Fox News hos...   
2  Russia-Ukraine war at a glance: what we know o...   
3  Takeaways from CNN’s town hall with Chris Chri...   
4  The different ways Republicans defend Trump ov...   

                                                Link   
0  https://news.google.com/rss/articles/CBMiY2h0d...  \
1  https://news.google.com/rss/articles/CBMiaWh0d...   
2  https://news.google.com/rss/articles/CBMidGh0d...   
3  https://news.google.com/rss/articles/CBMiWWh0d...   
4  https://news.google.com/rss/articles/CBMiMWh0d...   

                                         Description  
0  <ol><li><a href="https://news.google.com/rss/a...  
1  <ol><li><a href="https://news.google.com/rss/a...  
2  <ol><li><a href="https://news.google.com/rss/a...  
3  <ol><li><a href="https://news.google.com/rss/a...  
4  <ol><li><a href="https://news.google.com/rss/a...

# Apply Unsupervised clustering algorithm to get the labelled data

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
from sklearn.cluster import KMeans

In [48]:
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(df["Description"])

In [50]:
k=5
kmeans=KMeans(n_clusters=k,random_state=42)
kmeans.fit(X)

/home/student/.conda/envs/Mallikarjuna/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=5, random_state=42)

In [51]:
labels=kmeans.labels_

In [52]:
labels

array([2, 3, 0, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [56]:
df["Cluster"]=labels

In [57]:
cluster_counts=df["Cluster"].value_counts().sort_index()

In [58]:
cluster_counts

Cluster
0    28
1     2
2     3
3     2
4     1
Name: count, dtype: int64